# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902031


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:41,  1.44s/it]

Rendering models:  10%|▉         | 3/31 [00:03<00:29,  1.07s/it]

Rendering models:  13%|█▎        | 4/31 [00:03<00:21,  1.25it/s]

Rendering models:  16%|█▌        | 5/31 [00:07<00:49,  1.89s/it]

...

Rendering models:  81%|████████  | 25/31 [00:12<00:01,  4.30it/s]

Rendering models:  90%|█████████ | 28/31 [00:12<00:00,  5.46it/s]

Rendering models:  94%|█████████▎| 29/31 [00:12<00:00,  5.40it/s]

Rendering models:  97%|█████████▋| 30/31 [00:12<00:00,  5.02it/s]

equidad1                              0.000287
not-logged-in-8c0c44faf754d344b18a    0.000274
CTidwell3                             0.000298
ElisabethB                            0.000361
not-logged-in-0b0736c8991a7cc2017a    0.000687
not-logged-in-587a4e9ad373b8ff5983    0.000344
Drew_Farnsworth                       0.000292
Thedunkmasta                          0.001056
DrJackie                              0.000254
BirdLeaf                              0.000252
Alexander00                           0.000390
lsautter                              0.000337
not-logged-in-5f30b2bb9b7f9d49da41    0.000300
not-logged-in-09330e0b15263888ee9d    0.000731
gabrielweiss                          0.001241
Saharisunshine                        0.000264
pangeli5                              0.000358
farheenk88                            0.000348
rhondacard                            0.002700
Lavadude                              0.002071
not-logged-in-8700ff8251e0704f745c    0.000679
Denny2507    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())